In [ ]:
!pip install torch
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3 MB 30.7 MB/s 
     |████████████████████████████████| 2.8 MB 30.1 MB/s 
     |████████████████████████████████| 895 kB 34.0 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 636 kB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Load the tokenizer
from transformers import BertTokenizer, LineByLineTextDataset

vocab_file_dir = '/content/vocab.txt' 

tokenizer = BertTokenizer(vocab_file_dir)
sentence = 'There are many spoons on the table.'

encoded_input = tokenizer.tokenize(sentence)
#print(encoded_input)
# print(encoded_input['input_ids'])

In [ ]:
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = '/content/sentences.txt',
    block_size = 128  # maximum sequence length
)

print('No. of lines: ', len(dataset))

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


No. of lines:  4087


In [ ]:
test_dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = '/content/tes_sentences.txt',
    block_size = 128  # maximum sequence length
)

print('No. of lines: ', len(test_dataset))

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


No. of lines:  1295


In [ ]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=50000,
    hidden_size=768, 
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=512
)
 
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

No of parameters:  81965648


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/working/',
    #evaluation_strategy = "epoch",
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)


In [ ]:
import numpy as np
!pip install datasets
import datasets
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

     |████████████████████████████████| 264 kB 29.6 MB/s 
     |████████████████████████████████| 119 kB 36.8 MB/s 
     |████████████████████████████████| 243 kB 49.4 MB/s 


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=test_dataset,
    #compute_metrics=compute_metrics,
    #prediction_loss_only=True,
)

In [ ]:
trainer.train()
trainer.save_model('/content/working/')

***** Running training *****
  Num examples = 4087
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 384


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/working/
Configuration saved in /content/working/config.json
Model weights saved in /content/working/pytorch_model.bin


In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1295
  Batch size = 8


Perplexity: 926.93
